In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import activations
import numpy as np
import matplotlib.pyplot as plt
import h5py
from keras import backend as K

#Codigo de configuracion del modelo
#Batch size hace referencia al numero de muestras que se van a propagar por la red neuronal.
#Se toman los primeros 100 datos de todo el conjunto de datos y se entrena la red. Despues se toman los siguientes 100 datos y se vuelve a entrenar.
batch_size = 100
#Nº de iteraciones
no_epochs = 5
learning_rate = 0.007
#En problemas de clasificación la ultima capa tendra un numero de neuronas igual al numero de clases que existan en los datos.
no_classes = 10
#Tamaño del set de validacion, en este caso los datos de prueba son el 20% del total de datos
validation_split = 0.2
#Modo de reporte de avance(verbosity):Puede ser entero {0, 1, 2};  0 = silencio, 1 = barra de progreso, 2 = linea por época.
verbosity = 1

# Convierte vectores unidimensionales a valores 3D
# https://www.kaggle.com/daavoo/3d-mnist
def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:,:-1]

# Cambia la forma de los datos a un formato que se pueda manejar con capas Conv3D.
def rgb_data_transform(data):
    data_t = []
    for i in range(data.shape[0]):
        data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
    return np.asarray(data_t, dtype=np.float32)

# Para cargar el archivo de datos HDF5
with h5py.File("./full_dataset_vectors.h5", "r") as hf:    

    #Dividimos el conjunto del dataset en train y test
    X_train = hf["X_train"][:]
    targets_train = hf["y_train"][:]
    X_test = hf["X_test"][:] 
    targets_test = hf["y_test"][:]

    #Numero de datos de muestra
    sample_shape = (16, 16, 16, 3)

    # Cambia la forma de los datos a un formato 3D
    X_train = rgb_data_transform(X_train)
    X_test = rgb_data_transform(X_test)

    # Convierte los vectores objetivo (y_train, y_test) a objetivos categoricos
    #Al usar to_categorical(), un array numpy (o un vector) con numeros enteros que representan diferentes categorias,
    # pueden convertirse en un array numpy array (o en una matriz) que tendrá valores binarios y tantas columnas como numero de categories hay en los datos.
    targets_train = np.int64(to_categorical(targets_train))
    targets_test = to_categorical(targets_test).astype(np.int64)
    
    #Se define la funcion de activacion custom
    def custom_activation(x):
        return (K.sigmoid(x) * 5) - 1

    
    # Creacion del modelo
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation=tf.keras.activations., kernel_initializer='he_uniform', input_shape=sample_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    #model.add(Conv3D(64, kernel_size=(3, 3, 3), activation=custom_activation, kernel_initializer='he_uniform'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(no_classes, activation='softmax'))

    # Compilacion del modelo
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(learning_rate),
                metrics=['accuracy'])

    #Ajuste de los datos al modelo
    history = model.fit(X_train, targets_train,
                batch_size=batch_size,
                epochs=no_epochs,
                verbose=verbosity,
                validation_split=validation_split)

    # Aqui evaluamos como ha funcionado el modelo
    score = model.evaluate(X_test, targets_test, verbose=0)
    print(f'Pérdida: {score[0]} / Exactitud: {score[1]}')

#Entropía cruzada Categórica:Su propósito es tomar las probabilidades de salida (P) y medir la distancia desde los valores de verdad (resultado esperado).
# Suele ser adecuada en modelos de redes cuya salida representa una probabilidad,como cuando hacemos una clasificación categórica con función de activación ‘softmax’. 
# Se puede utilizar, por ejemplo, en problemas de clasificación categórica con una sola etiqueta de salida y precedida de una función de activación ‘Softmax’.
    
    
    
    # Representacion : Entropía cruzada Categórica y exactitud
    plt.plot(history.history['loss'], label='Entropía cruzada categórica(datos de entrenamiento)')
    plt.plot(history.history['val_loss'], label='Entropía cruzada categórica (datos de validación)')
    plt.plot(history.history['accuracy'], label='Exactitud (datos de entrenamiento)')
    plt.plot(history.history['val_accuracy'], label='Exactitud(datos de validación)')
    plt.title('Funcionamiento del modelo con el dataset 3D MNIST')
    plt.ylabel('Pérdida o error')
    plt.xlabel('Numero de iteraciones')
    plt.legend(loc="upper left")
    plt.show()


Epoch 1/5
80/80 [==============================] - 68s 842ms/step - loss: 5.9982 - accuracy: 0.2027 - val_loss: 2.0698 - val_accuracy: 0.3325
Epoch 2/5
80/80 [==============================] - 67s 838ms/step - loss: 1.8779 - accuracy: 0.3932 - val_loss: 1.7908 - val_accuracy: 0.4450
Epoch 3/5
 3/80 [>.............................] - ETA: 1:03 - loss: 1.6740 - accuracy: 0.4967

https://www.tensorflow.org/api_docs/python/tf/keras/activations